In [1]:
from src import notebook, datasets, build_models,caluclate_basis, custom_blocks, train, utils, averaging
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import wandb
import torchvision.transforms as transforms
from tqdm import tqdm   
from torchsummary import summary

In [2]:
def make(config):

    train_loader, val_loader, test_loader = datasets.get_dataloaders(config, logfile=None, summaryfile=None, log=False) 

    model = build_models.build_model_from_args(config)

    if config.criterion == "CrossEntropyLoss":
        criterion = nn.CrossEntropyLoss()
    elif config.criterion == "MSELoss":
        criterion = nn.MSELoss()
    else:
        raise ValueError(f"Criterion {criterion} not supported")
    
    if config.optimizer == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config.lr)
    elif config.optimizer == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=config.lr)
    else:
        raise ValueError(f"Optimizer {optimizer} not supported")
    
    return model, train_loader, val_loader, test_loader, criterion, optimizer

In [3]:
hyperparameters = dict(arch=['64', 'M', '128', 'M'], 
                                 batch_norm=False,
                                 bias=True,
                                 classifier_layers=[256],
                                 classifier_bias= True,
                                 classifier_dropout=0,
                                 avgpool=False,
                                 avgpool_size=[1,1],  # Adjusted avgpool size to (1, 1)
                                 dataset="mnist",
                                 input_height = 29,
                                 input_width = 29,
                                 greyscale=True,
                                 n_classes= 10,
                                 epochs=10, 
                                 batch_size=64,
                                 optimizer="Adam",
                                 weight_normalization=True,
                                 lr=0.001,
                                 criterion="CrossEntropyLoss",
                                 seed=42,
                                 save_checkpoints=False,
                                 save_dir = "",
                                 checkpoint_type = "epoch",
                                 )

In [4]:
run = wandb.init(project="averaging-with-vgg-model", config=hyperparameters)
config = wandb.config

model, train_loader, val_loader, test_loader, criterion, optimizer = make(config)

print(model)

print(criterion, optimizer)

# and use them to train the model
train.train(config, model, train_loader, val_loader, test_loader,criterion, optimizer)


#rotate the test set
test_loader = averaging.random_rotate_dataset(test_loader)

train.test_model(model, test_loader, device="cuda")

# and evaluate the model

averaging.average_over90degrees_and_evaluate(model, test_loader, device="cuda")

#finish run
run.finish()

wandb: Currently logged in as: atsou2 (atsou2-johns-hopkins-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


CustomModel(
  (features): Sequential(
    (0): ParametrizedConv2d(
      1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): _WeightNorm()
        )
      )
    )
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ParametrizedConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): _WeightNorm()
        )
      )
    )
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): CustomClassifier(
    (classifier_layers): Sequential(
      (0): Sequential(
        (0): Linear(in_features=6272, out_features=256, bias=True)
        (1): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Linear(in_features=256, out_features=10, bias=True

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, Examples seen: 640, Loss: 1.2896
Epoch 0, Examples seen: 1280, Loss: 1.2564
Epoch 0, Examples seen: 1920, Loss: 1.2692
Epoch 0, Examples seen: 2560, Loss: 1.1066
Epoch 0, Examples seen: 3200, Loss: 1.1233
Epoch 0, Examples seen: 3840, Loss: 1.1630
Epoch 0, Examples seen: 4480, Loss: 0.8945
Epoch 0, Examples seen: 5120, Loss: 0.9907
Epoch 0, Examples seen: 5760, Loss: 1.1783
Epoch 0, Examples seen: 6400, Loss: 1.1184
Epoch 0, Examples seen: 7040, Loss: 0.8697
Epoch 0, Examples seen: 7680, Loss: 0.9218
Epoch 0, Examples seen: 8320, Loss: 0.9798
Epoch 0, Examples seen: 8960, Loss: 1.0259
Epoch 0, Examples seen: 9600, Loss: 1.0623
Epoch 0, Examples seen: 10240, Loss: 1.2587
Epoch 0, Examples seen: 10880, Loss: 1.0381
Epoch 0, Examples seen: 11520, Loss: 1.1153
Epoch 0, Examples seen: 12160, Loss: 1.0526
Epoch 0, Examples seen: 12800, Loss: 1.2391
Epoch 0, Examples seen: 13440, Loss: 1.1174
Epoch 0, Examples seen: 14080, Loss: 1.2194
Epoch 0, Examples seen: 14720, Loss: 0.8442
Epoc

 10%|█         | 1/10 [00:53<08:04, 53.83s/it]

...

100%|██████████| 157/157 [00:11<00:00, 13.99it/s]


Test Accuracy: 10.84%


100%|██████████| 157/157 [00:29<00:00,  5.25it/s]

Test Accuracy: 9.76%


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
loss,▅▆▁▇▄▂▆▆▂▅▃▂▄▂▆▆▅▁▅▆▄▃▆▅▇▆▆▇▆▇▅▃█▄▆▆▆▆▂▅
test_accuracy,▁
test_accuracy_after_averaging,▁
train_accuracy,▁▆▇▇▇▇████
train_loss,█▃▂▂▁▁▁▁▁▁
val_accuracy,▁▇▆▆▆▅▇▆▆█
epoch,9
loss,0.82751
test_accuracy,10.84
test_accuracy_after_averaging,9.76


In [5]:

# Load ResNet-18 model
def load_resnet18(n_classes):
    model = models.resnet18(pretrained=True)
    # Modify the final fully connected layer to have n_classes output units
    model.fc = nn.Linear(model.fc.in_features, n_classes)
    return model